In [1]:
"""
Demo low-performance EPI sequence without ramp-sampling.
"""

import numpy as np

import pypulseq as pp
import matplotlib
import MRzeroCore as mr0
import matplotlib.pyplot as plt
import torch
from seq_utils import plot_signal

matplotlib.use('TkAgg')

In [2]:
torch.cuda.is_available()

True

In [3]:
# plot = True
# write_seq = True
# seq_filename = 'epi_pypulseq.seq'
# # ======
# # SETUP
# # ======
# R=1
# seq = pp.Sequence()  # Create a new sequence object
# # Define FOV and resolution
# fov = 220e-3
# Nx = 256
# Ny = 256 * 1//R
# slice_thickness = 3e-3  # Slice thickness
# n_slices = 1
# 
# # Set system limits
# system = pp.Opts(
#     max_grad=32,
#     grad_unit='mT/m',
#     max_slew=130,
#     slew_unit='T/m/s',
#     rf_ringdown_time=30e-6,
#     rf_dead_time=100e-6,
# )
# 
# # ======
# # CREATE EVENTS
# # ======
# # Create 90 degree slice selection pulse and gradient
# rf, gz, _ = pp.make_sinc_pulse(
#     flip_angle=np.pi / 2,
#     system=system,
#     duration=3e-3,
#     slice_thickness=slice_thickness,
#     apodization=0.5,
#     time_bw_product=4,
#     return_gz=True,
# )
# 
# # Define other gradients and ADC events
# delta_k = 1 / fov
# k_width = Nx * delta_k
# dwell_time = 4e-6
# readout_time = Nx * dwell_time
# flat_time = np.ceil(readout_time * 1e5) * 1e-5  # round-up to the gradient raster
# gx = pp.make_trapezoid(
#     channel='x',
#     system=system,
#     amplitude=k_width / readout_time,
#     flat_time=flat_time,
# )
# adc = pp.make_adc(
#     num_samples=Nx,
#     duration=readout_time,
#     delay=gx.rise_time + flat_time / 2 - (readout_time - dwell_time) / 2,
# )
# 
# # Pre-phasing gradients
# pre_time = 8e-4
# gx_pre = pp.make_trapezoid(channel='x', system=system, area=-gx.area / 2, duration=pre_time)
# gz_reph = pp.make_trapezoid(channel='z', system=system, area=-gz.area / 2, duration=pre_time)
# gy_pre = pp.make_trapezoid(channel='y', system=system, area=-Ny / 2 * delta_k, duration=pre_time)
# 
# # Phase blip in the shortest possible time
# dur = np.ceil(2 * np.sqrt(delta_k / system.max_slew) / 10e-6) * 10e-6
# gy = pp.make_trapezoid(channel='y', system=system, area=delta_k, duration=dur*R)
# 
# # ======
# # CONSTRUCT SEQUENCE
# # ======
# # Define sequence blocks
# for s in range(n_slices):
#     rf.freq_offset = gz.amplitude * slice_thickness * (s - (n_slices - 1) / 2)
#     seq.add_block(rf, gz)
#     seq.add_block(gx_pre, gy_pre, gz_reph)
#     for i in range(Ny):
#         seq.add_block(gx, adc)  # Read one line of k-space
#         seq.add_block(gy)  # Phase blip
#         gx.amplitude = -gx.amplitude  # Reverse polarity of read gradient
# 
# ok, error_report = seq.check_timing()
# if ok:
#     print('Timing check passed successfully')
# else:
#     print('Timing check failed! Error listing follows:')
#     print(error_report)
# 
# 
# # =========
# # WRITE .SEQ
# # =========
# if write_seq:
#     seq.write(seq_filename)

Timing check passed successfully


In [6]:
plot = True
write_seq = True
seq_filename = 'epi_pypulseq.seq'
# ======
# SETUP
# ======
R=1
seq = pp.Sequence()  # Create a new sequence object
# Define FOV and resolution
fov = 220e-3
Nx = 64
Ny = int(64 * 1/R)
TE = 10e-3
use_partial_fourier = False
if use_partial_fourier:
    Ny *= 9/16
Ny = int(Ny)

slice_thickness = 3e-3  # Slice thickness
n_slices = 1

# Set system limits
system = pp.Opts(
    max_grad=32,
    grad_unit='mT/m',
    max_slew=130,
    slew_unit='T/m/s',
    rf_ringdown_time=30e-6,
    rf_dead_time=100e-6,
)

# ======
# CREATE EVENTS
# ======
# Create 90 degree slice selection pulse and gradient
rf, gz, _ = pp.make_sinc_pulse(
    flip_angle=np.pi / 2,
    system=system,
    duration=3e-3,
    slice_thickness=slice_thickness,
    apodization=0.5,
    time_bw_product=4,
    return_gz=True,
)

# Define other gradients and ADC events
delta_k = 1 / fov
k_width = Nx * delta_k
dwell_time = 4e-6
readout_time = Nx * dwell_time
flat_time = np.ceil(readout_time * 1e5) * 1e-5  # round-up to the gradient raster
gx = pp.make_trapezoid(
    channel='x',
    system=system,
    amplitude=k_width / readout_time,
    flat_time=flat_time,
)
adc = pp.make_adc(
    num_samples=Nx,
    duration=readout_time,
    delay=gx.rise_time + flat_time / 2 - (readout_time - dwell_time) / 2,
)

# Pre-phasing gradients
pre_time = 32e-4
gx_pre = pp.make_trapezoid(channel='x', system=system, area=-gx.area / 2, duration=pre_time)
gz_reph = pp.make_trapezoid(channel='z', system=system, area=-gz.area / 2, duration=pre_time)
gy_pre = pp.make_trapezoid(channel='y', system=system, area=-Ny / 2 * delta_k, duration=pre_time)

print(f"pre y: {-Ny / 2 * delta_k}")
print("delta_k: ", delta_k)
print("Ny: ", Ny)
# Phase blip in the shortest possible time
dur = np.ceil(2 * np.sqrt(delta_k / system.max_slew) / 10e-6) * 10e-6
gy = pp.make_trapezoid(channel='y', system=system, area=delta_k, duration=dur*R)

# ======
# CONSTRUCT SEQUENCE
# ======
# Define sequence blocks
for s in range(n_slices):
    rf.freq_offset = gz.amplitude * slice_thickness * (s - (n_slices - 1) / 2)
    seq.add_block(rf, gz)
    seq.add_block(gx_pre, gy_pre, gz_reph)
    for i in range(Ny):
        seq.add_block(gx, adc)  # Read one line of k-space
        seq.add_block(gy)  # Phase blip
        gx.amplitude = -gx.amplitude  # Reverse polarity of read gradient

ok, error_report = seq.check_timing()
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed! Error listing follows:')
    print(error_report)


# =========
# WRITE .SEQ
# =========
if write_seq:
    seq.write(seq_filename)
if plot:
    seq.plot()

pre y: -145.45454545454547
delta_k:  4.545454545454546
Ny:  64
Timing check passed successfully


In [4]:
seq = mr0.Sequence.import_file(seq_filename)
seq.plot_kspace_trajectory()

In [7]:
# signal = mr0.util.simulate(seq.cuda())
signal = mr0.util.simulate_2d(seq)

DEPRECATED: util.simulate_2d will be removed in the future.
Use util.simulate() instead (together with util.load_phantom() if necessary
Calculating repetition 1 / 1 - done


In [10]:
signal.shape

torch.Size([4096, 1])

In [ ]:
signal.shape[0] / 256

In [15]:
# Calculate actual acquired lines
lines_acquired = Ny

# ======
# VISUALIZATION
# ======
plot = True
if plot:
    signal = mr0.util.simulate_2d(seq)
    seq.plot(plot_now=False)
    mr0.util.insert_signal_plot(seq=seq, signal=signal.numpy())
    plt.show()
    plot_signal(signal, lines_acquired, Nx, R)
    print("hello")

DEPRECATED: util.simulate_2d will be removed in the future.
Use util.simulate() instead (together with util.load_phantom() if necessary
Calculating repetition 1 / 1 - done
hello


In [ ]:
# from partial_fourier_recon import pocs_pf
